In [13]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import rpy2

In [3]:
article_id = 14226968  
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26844650,
  'name': 'allyears.csv.zip',
  'size': 2405908113,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26844650',
  'supplied_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'computed_md5': '9e046ac05ecd2c32a256a47dd1098b81'},
 {'id': 26863682,
  'name': 'individual_years.zip',
  'size': 1896206676,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26863682',
  'supplied_md5': '921da748974b07b2a70bbfcc04535a77',
  'computed_md5': '921da748974b07b2a70bbfcc04535a77'},
 {'id': 27515426,
  'name': 'combined_model_data.csv.zip',
  'size': 821308997,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/27515426',
  'supplied_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'computed_md5': '7638434c44a7d29cbb29fe200b4fd65d'},
 {'id': 27520682,
  'name': 'combined_model_data_parti.parquet.zip',
  'size': 519743915,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/

In [5]:
%%time
files_to_dl = ["individual_years.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 8.84 s, sys: 21.2 s, total: 30.1 s
Wall time: 3min 42s


In [7]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "individual_years.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 38 s, sys: 16.9 s, total: 54.9 s
Wall time: 1min 25s


In [8]:
%%time
import pandas as pd
use_cols = ["ArrDelay", "DepDelay", "Distance", "TailNum","UniqueCarrier","Origin","Dest"]
files = glob.glob('figshareairline/individual_years/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(year=re.findall(r'[^\/]+(?=\.)', file)[0])
                for file in files)
              )
df.to_csv("figshareairline/combined_data.csv")

CPU times: user 5min 44s, sys: 42.6 s, total: 6min 26s
Wall time: 6min 59s


In [11]:
df.to_parquet("figshareairline/combined_data.parquet")

In [15]:
%%sh
du -sh figshareairline/combined_data.csv
du -sh figshareairline/combined_data.parquet

3.1G	figshareairline/combined_data.csv
417M	figshareairline/combined_data.parquet


Parquet successfully reduced `combined_data.csv` from 3.1G to `combined_data.parquet` with 417M.

<br>

## Load dataframe through parquet in python

In [21]:
%%time
df_py = pd.read_parquet('figshareairline/combined_data.parquet', 
                        filters=[('year', '=', '1996'), ('Origin', '=', 'ATL')],
                       columns = ['UniqueCarrier', 'DepDelay'])
print(df_py.groupby('UniqueCarrier').mean('DepDelay'))

                DepDelay
UniqueCarrier           
AA              8.763666
CO             10.283951
DL             12.411944
HP             23.552827
NW              6.363534
TW              7.770502
UA             12.685468
US             11.438999
CPU times: user 5.31 s, sys: 1.98 s, total: 7.29 s
Wall time: 7.87 s


In python it takes total 7.29s to load and perform the computation.

<br>
load dataframe through parquet in R

In [15]:
%load_ext rpy2.ipython

In [35]:
%%time
%%R
suppressMessages(library(dplyr))
suppressMessages(library(arrow))
df_r <- open_dataset('figshareairline/combined_data.parquet') |>
    filter(year == 1996, Origin == 'ATL') |>
    select(UniqueCarrier, DepDelay) |>
    group_by(UniqueCarrier) |>
    summarise(avg = mean(DepDelay))



CPU times: user 50.2 ms, sys: 4.13 ms, total: 54.3 ms
Wall time: 64.4 ms


in r it takes total 44.2ms to load and perform the computation, much faster! thanks to the lazy evaluation. 